## The Reddit Climate Change Dataset

### Mounting the drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Libraries

In [1]:
import csv
import time
import random

In [2]:
!pip install pytest pytest-sugar

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Dataset path

In [3]:
path = '/content/drive/MyDrive/dataset/the-reddit-climate-change-dataset-comments.csv'

### Function 1

Given an id of a message on Reddit, return all information about the message.


In [4]:
class Climate():                     
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        self.id_to_row = {}                        
        for row in self.rows:                 
            self.id_to_row[row[1]] = row 
        
    def get_message_from_id(self, id):   
        '''
        Description: this function will look over the dataset
        for some line with the id value passed. If there is any
        line with that id, it will return the row, else, it 
        will return None

        Parameters:
        - id <int>: value of the id we want to search

        Return: it returns the row with the respective id, or 
        returns none if there isn't this id in the dataset
        '''
        for row in self.rows:                 
            if row[1] == id:
                return row
        return None

    def get_message_from_id_fast(self, id):   
        '''
        Description: this function will do a lookup in a dictionary 
        to check if any dictionary key matches the id. If there is 
        a match, it will return the row, else, it will return None

        Parameters:
        - id <int>: value of the id we want to search

        Return: it returns the row with the respective id, or 
        returns none if there isn't this id in the dataset
        '''
        if id in self.id_to_row:            
            return self.id_to_row[id]
        return None  

    def get_data(self):
        '''
        Description: this function returns all the rows of the dataset
        '''
        return self.rows  

In [5]:
climate = Climate(path) 
print(climate.get_message_from_id('imlc8w9')) 
print(climate.get_message_from_id('abcd1234')) 
print(climate.get_message_from_id_fast('imlc8w9')) 
print(climate.get_message_from_id_fast('abcd1234')) 

['comment', 'imlc8w9', '2tja6', 'terriblefacebookmemes', 'false', '1661989862', 'https://old.reddit.com/r/terriblefacebookmemes/comments/x1p4ro/its_complicated/imlc8w9/', 'It doesn’t matter what you define “real science.” Science is science. Ignoring what you don’t like doesn’t invalidate it. Doesn’t help you’re painting people with very real and serious concerns as alarmists\n\nAnd no, I’m not going to bother. I can already guess that any proof I put forward, you’re just going to link the Greenfacts.org article and continue to call me an alarmist, that “I’m causing a panic for no reason.” Despite, you know, climate change is already occurring and every effort to address it is blocked by oil barons like the Koch brothers, who sponsor reactionary think tanks to put out climate change denialist propaganda so they can continue milking as much money out of their buisness before they inevitably fail and move on to other projects with their blood money.', '-0.8238', '1']
None
['comment', 'im

#### Comparing performance

To compare the performance of the first main function, a list was created with all the ids of the dataset, and then 100 ids were randomly selected to be searched, and thus, the execution time of the functions `get_message_from_id` and `get_message_from_id_fast` was computed.

In [8]:
ids = [row[1] for row in climate.get_data()] 
ids_teste = [ids[random.randint(0, len(ids))] for _ in range(100)]

total_time_no_dict = 0                                           
for identifier in ids_teste:                                             
    start = time.time()                                            
    climate.get_message_from_id(identifier)                        
    end = time.time()                                             
    total_time_no_dict += end - start                              
    
total_time_dict = 0                                                 
for identifier in ids_teste:                                              
    start = time.time()                                         
    climate.get_message_from_id_fast(identifier)               
    end = time.time()                                          
    total_time_dict += end - start                            
    
print('Total time without using dict: ', round(total_time_no_dict, 6))                                      
print('Total time using dict: ', round(total_time_dict, 6))

Total time without using dict:  28.633346
Total time using dict:  0.000129


As we can see, the function `get_message_from_id_fast` (implemented with a dictonary) was much faster than the function `get_message_from_id` (implemented with `for`), in fact, it was 28.633346/0.000129 = **221963** times faster.

#### Test

In [4]:
%%file test_data.py
import pytest, csv

class Climate():                   
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        self.id_to_row = {}                        
        for row in self.rows:                 
            self.id_to_row[row[1]] = row 
        
    def get_message_from_id(self, id):   
        for row in self.rows:                 
            if row[1] == id:
                return row
        return None

    def get_message_from_id_fast(self, id):   
        if id in self.id_to_row:            
            return self.id_to_row[id]
        return None  

    def get_data(self):
      return self.rows  

@pytest.fixture(scope="session")
def data():
    climate = Climate('/content/drive/MyDrive/dataset/the-reddit-climate-change-dataset-comments.csv') 
    return climate

def test_find_id_get_message(data):
    """
    Test 1 - Look up a dataset row based on an id, knowing that the id exists in the dataset (without using dictonary)
    """
    assert data.get_message_from_id('imlc8w9') == ['comment', 'imlc8w9', '2tja6', 'terriblefacebookmemes', 'false', '1661989862', 'https://old.reddit.com/r/terriblefacebookmemes/comments/x1p4ro/its_complicated/imlc8w9/', 'It doesn’t matter what you define “real science.” Science is science. Ignoring what you don’t like doesn’t invalidate it. Doesn’t help you’re painting people with very real and serious concerns as alarmists\n\nAnd no, I’m not going to bother. I can already guess that any proof I put forward, you’re just going to link the Greenfacts.org article and continue to call me an alarmist, that “I’m causing a panic for no reason.” Despite, you know, climate change is already occurring and every effort to address it is blocked by oil barons like the Koch brothers, who sponsor reactionary think tanks to put out climate change denialist propaganda so they can continue milking as much money out of their buisness before they inevitably fail and move on to other projects with their blood money.', '-0.8238', '1']


def test_doesnt_find_id_get_message(data):
    """
    Test 2 - Look up a dataset row based on an id, knowing that the id doesn't exist in the dataset (without using dictonary)
    """
    assert data.get_message_from_id('abcd1234') == None

def test_find_id_get_message_fast(data):
    """
    Test 3 - Look up a dataset row based on an id, knowing that the id exists in the dataset (using dictonary)
    """
    assert data.get_message_from_id_fast('imlc8w9') == ['comment', 'imlc8w9', '2tja6', 'terriblefacebookmemes', 'false', '1661989862', 'https://old.reddit.com/r/terriblefacebookmemes/comments/x1p4ro/its_complicated/imlc8w9/', 'It doesn’t matter what you define “real science.” Science is science. Ignoring what you don’t like doesn’t invalidate it. Doesn’t help you’re painting people with very real and serious concerns as alarmists\n\nAnd no, I’m not going to bother. I can already guess that any proof I put forward, you’re just going to link the Greenfacts.org article and continue to call me an alarmist, that “I’m causing a panic for no reason.” Despite, you know, climate change is already occurring and every effort to address it is blocked by oil barons like the Koch brothers, who sponsor reactionary think tanks to put out climate change denialist propaganda so they can continue milking as much money out of their buisness before they inevitably fail and move on to other projects with their blood money.', '-0.8238', '1']

def test_doesnt_find_id_get_message_fast(data):
    """
    Test 4 - Look up a dataset row based on an id, knowing that the id doesn't exist in the dataset (using dictonary)
    """
    assert data.get_message_from_id_fast('abcd1234') == None

Overwriting test_data.py


In [5]:
!pytest . -vv

Test session starts (platform: linux, Python 3.7.14, pytest 3.6.4, pytest-sugar 0.9.5)
cachedir: .pytest_cache
rootdir: /content, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_data.py::test_find_id_get_message ✓                         25% ██▌       
 test_data.py::test_doesnt_find_id_get_message ✓                  50% █████     
 test_data.py::test_find_id_get_message_fast ✓                    75% ███████▌  
 test_data.py::test_doesnt_find_id_get_message_fast ✓            100% ██████████

Results (151.08s):
       4 passed


### Function 2

Given a lower and upper bound of the "sentiment" column, return all messages with sentiment values between the lower and upper bounds.

In [ ]:
def row_sentiment(row):
  return row[-2]

class Climate():                   
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for r in self.rows:
          if r[-2] == '' or r[-2] == ' ': 
            r[-2] = 0.0
          else:
            r[-2] = float(r[-2])
        self.rows_by_sentiment = sorted(self.rows, key=row_sentiment)

    def find_message_with_sentiment(self, sentiment, r_start):
      '''
      Description: this function does a binary search through the dataset
      based on the value of "sentiment" column. If the value of the sentiment
      is in the dataset, it returns the value of the index of the column, but
      if the value isn't in the dataset, it should return -1

      Parameters:
      - sentiment <float>: represents the value of the sentiment
      - r_start <int>: represents the range_start of the binary search

      Return:
      This functions returns the index of the column with the value of the
      sentiment or the value -1 if there isn't any row in the dataset with
      the value of the sentiment
      '''
      range_start = r_start                                   
      range_end = len(self.rows_by_sentiment) - 1                       
      while range_start < range_end:
          range_middle = (range_end + range_start) // 2  
          value = self.rows_by_sentiment[range_middle][-2]
          if value == sentiment:                            
              return range_middle                        
          elif value < sentiment:                           
              range_start = range_middle + 1             
          else:                                          
              range_end = range_middle - 1 
      if self.rows_by_sentiment[range_start][-2] != sentiment:                  
          return -1                                      
      return range_start

    def messages_between_sentiments(self, inf, sup):
      '''
      Description: this function takes two values, the inferior limit and the
      superior limit of the sentiment variable values, and it returns all the
      messages with sentiment values between these limits. If there isn't any
      row with the value of the inferior limit or the superior limit, it should
      return -1

      Parameters:
      - inferior_limit <int>: represents the inferior limit of the sentiment
      column value
      - superior_limit <int>: represents the superior limit of the sentiment
      column value

      Return:
      This function returns the messages with sentiment values between the limits,
      but, if there isn't any row with the value of the inferior limit or the 
      superior limit, it returns -1
      '''
      inferior_limit = self.find_message_with_sentiment(inf, 0)
      if inferior_limit == -1:
        return -1
      superior_limit = self.find_message_with_sentiment(sup, inferior_limit)
      if superior_limit == -1:
        return -1
      data = self.rows_by_sentiment[inferior_limit:superior_limit+1]
      return [row[-3] for row in data] 

In [ ]:
climate = Climate(path) 
print(climate.messages_between_sentiments(-0.2889, -0.2888))
print(climate.messages_between_sentiments(-0.28891234, -0.2888))
print(climate.messages_between_sentiments(-0.2889, -0.28881234))

['Yeah, these people don\'t give a shit about the poor or curbing wasteful consumption. I talked to a supposed "AnCom" who was oh so concerned about climate change and so sure that capitalism was ruining the planet and making people poor up until it came around to him giving up various comforts. No, you see, those are necessities to the communist! Fucking everything that they want is a necessity because it\'s theirs. Your shit on the other hand should be theirs.\n\nOnce again, their ideology is no more than "GIBS ME DAT!" The lowest of low IQ ideologies out there.', 'Just a quick question. Why deny climate change when the evidence for a global rise in temperature, melting ice caps, rising ocean levels and other such things is so prominent? I mean the data is literally right in front of you, yet you deny it. Why?', "Why not climate change? Isn't that more urgent than we expected?", 'the US can, sure. Maybe some of Europe. but what about Canada, Australia, Japan? Or the rapidly developin

#### Test

In [ ]:
%%file test_data.py
import pytest, csv

def row_sentiment(row):
  return row[-2]

class Climate():                   
    
    def __init__(self, csv_filename): 
        with open(csv_filename) as f: 
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]
        for r in self.rows:
          if r[-2] == '' or r[-2] == ' ': 
            r[-2] = 0.0
          else:
            r[-2] = float(r[-2])
        self.rows_by_sentiment = sorted(self.rows, key=row_sentiment)

    def find_message_with_sentiment(self, sentiment, r_start):
      range_start = r_start                                   
      range_end = len(self.rows_by_sentiment) - 1                       
      while range_start < range_end:
          range_middle = (range_end + range_start) // 2  
          value = self.rows_by_sentiment[range_middle][-2]
          if value == sentiment:                            
              return range_middle                        
          elif value < sentiment:                           
              range_start = range_middle + 1             
          else:                                          
              range_end = range_middle - 1 
      if self.rows_by_sentiment[range_start][-2] != sentiment:                  
          return -1                                      
      return range_start

    def messages_between_sentiments(self, inf, sup):
      inferior_limit = self.find_message_with_sentiment(inf, 0)
      if inferior_limit == -1:
        return -1
      superior_limit = self.find_message_with_sentiment(sup, inferior_limit)
      if superior_limit == -1:
        return -1
      data = self.rows_by_sentiment[inferior_limit:superior_limit+1]
      return [row[-3] for row in data] 

@pytest.fixture(scope="session")
def data():
    climate = Climate('/content/drive/MyDrive/dataset/the-reddit-climate-change-dataset-comments.csv') 
    return climate

def test_without_inferior_limit(data):
    """
    Test 1
    """
    assert data.messages_between_sentiments(-0.2889121212, -0.2888) == -1


def test_without_superior_limit(data):
    """
    Test 2
    """
    assert data.messages_between_sentiments(-0.2889, -0.2888647328) == -1

def test_with_both_limits(data):
    """
    Test 3
    """
    assert data.messages_between_sentiments(-0.2889, -0.2888) == ['Yeah, these people don\'t give a shit about the poor or curbing wasteful consumption. I talked to a supposed "AnCom" who was oh so concerned about climate change and so sure that capitalism was ruining the planet and making people poor up until it came around to him giving up various comforts. No, you see, those are necessities to the communist! Fucking everything that they want is a necessity because it\'s theirs. Your shit on the other hand should be theirs.\n\nOnce again, their ideology is no more than "GIBS ME DAT!" The lowest of low IQ ideologies out there.', 'Just a quick question. Why deny climate change when the evidence for a global rise in temperature, melting ice caps, rising ocean levels and other such things is so prominent? I mean the data is literally right in front of you, yet you deny it. Why?', "Why not climate change? Isn't that more urgent than we expected?", 'the US can, sure. Maybe some of Europe. but what about Canada, Australia, Japan? Or the rapidly developing countries which will soon eclipse the states in power needs? India, China, South America, Africa?\n\ncan they all switch in 5 to 7 years. absolutely not. no chance. itll be 50+ years and guess what, climate change is a global problem, not a US problem. ignore current output levels: think about future greenhouse gas emissions.\n\n', "no, the research done in america is just some of the most advanced in the world at the moment. America itself is not doing very well in the combating of climate change itself though. compared to other countries such as Finland, Germany, The Netherlands and countless others are actually investing into using those technologies. Its great that america is doing the research but if they dont use it or invest into making it plausible its useless and wont stay there long. Even China is investing billions into renewable while america is one of the only countries in the world that is decreasing their investments. You were leading because of Obama and others. Funny how you're arguing against someone with something they helped establish.", '[Explain this then, know-it-all. How is this not more important than what climate change is "costing"? As if the fucking money even matters](https://www.nytimes.com/2017/03/15/science/great-barrier-reef-coral-climate-change-dieoff.html?_r=0) ', "I never understood the population control point of view. Human beings are literally the only thing that matters. Climate change is a big deal because it threatens us. \n\nPeople aren't the most important resource, they are the *only* resource. \n\nWhat are the chances of another Einstein being born? Someone who will single handedly expand our understanding of the universe? 1 in a billion? Less? \n\nHumanity needs more people on the extreme right side of the bell curve, not less. ", "That's an interesting theory. They ran an experiment, subjected an experimental group to famine, a control group to no famine, and always found that ISIS started shit with the experimental group?\n\nI suppose that's what's so special about climatology... no one ever needs to prove anything. One more trailer park than usual tornadoed in Alabama? Must be climate change.", '&gt; There is no null hypothesis with regard to one\'s belief on the question of god\'s existence, which is what the theism/atheism debate is mainly concerned with.\n\nThere is a default position on absolutely everything. That position is: I don\'t believe in anything you say but I don\'t know for sure. The other options are:\n\n* to believe anything someone says, which is being gullible and is not how you get knowledge, or\n* to know something for sure when you don\'t know anything about it, which is just lying\n\nSo, agnostic atheism - not believing in Gods but not knowing - is the default.\n\n&gt; there are some gods of which you\'ve never heard\n\nNote: I never heard of any Gods growing up, so I didn\'t know about - and didn\'t believe in - any Gods. It was only much later that I was introduced to Gods and religions and I didn\'t believe their claims.\n\n&gt; Your atheism falls on a spectrum ...\n\nI think this is fair to say. Is there a problem with this view?\n\n&gt; I think most people are concerned with creator gods\n\nWell in the US or this sub perhaps. I\'m a lot more interested in the Greek Gods myself but I get your point :]\n\n&gt; observation can never demonstrate that god, in general, does not exist\n\nSure though it\'s awfully inconvenient - we used to be able to see him and he seems to be MIA for a while now.\n\n&gt; you also have a belief that unicorns don\'t exist. Am I wrong?\n\nYou are. I have no proof that unicorns don\'t exist. It\'s possible that the actual animal (rather than its magical properties) did actually exist at one point. If you mean that I have a belief that the magical unicorns don\'t exist then I would simply say generally that magic doesn\'t exist and therefore magical unicorns, being a subset of magic, don\'t exist. The animal itself - as far as the horn - could.\n\n&gt; Well that\'s not entirely true. \n\nYes it is. If you\'re into Judaism or Christianity, God definitely used to appear and talk to you. Some people claim he still does. So, in the context of my typical debates with Christians, this behavior is expected.\n\n&gt; It\'s possible that appearing to us would sully our free will \n\nAgain, if the context is the Abrahamic God then this isn\'t an issue. For other Gods, perhaps. Here\'s the thing though: a God that doesn\'t make themselves known is no different from one that doesn\'t exist.\n\n&gt; "omnipotence," as a very specific philosophical term, has lately and often been hijacked by both bible-thumping idiots, as well as atheists constructing strawmen\n\nSpeaking for me personally, I go off of what theists describe. I wouldn\'t call them "bible-thumping idiots" though. They\'re the ones making the claims and I respond to those claims.\n\nThe thing about agnosticism is that there is a difference between believing something and knowing something. For example, I believe climate change is occuring. I have some reasons for this as far as I find it unlikely that all our pollution has no effect on the environment and the weather has been going nuts for at least a decade. But, I don\'t know for sure. I am too ignorant about the scientific data to know if what they\'re saying is correct or not. If I devoted a lot of time to it then I could know better but since it doesn\'t bother me much, I really don\'t care.\n\nHowever, this is a bad example because if religions are true and/or Gods exist then this should be the most important thing to know about. It\'s not as trivial as climate change.\n\nI separate belief and knowledge. To me, knowledge is verified belief - that not only do others believe it but you can prove it. Belief is unproven knowledge. It might be true to you but that\'s not knowledge, it\'s a personal opinion.\n\n&gt; If one has justifications for believing something, and that belief just so happens to be true, that person has knowledge.\n\nThe key phrase is: that belief just so happens to be true. Without it, it\'s a belief. All belief in any Gods is not knowledge because it fails the "just so happens to be true" part.\n\nMind you, I might disagree with your definitions. After all, it\'s my belief - not knowledge - that philosophy has proven exactly nothing in its entire length of existence.\n\n&gt; You only believe something because you think it is true.\n\nSure... it just doesn\'t make it true.', "So what do you think a minister of climate change should be doing exactly?  What roles will he/she be doing and how much time and money is needed to perform these tasks?  If you are so sure that there needs to be one, you surely have a list of duties to be performed that can't possibly be covered by the new set-up.\n\nTransparency?  What is the problem here?  You have a list of ministers and a list of their responsibilities.  Can't get more transparent than that.  You think there is some conspiracy going on, don't you?", 'Chinese use ac for heating as well. Additionally most of their energy is from coal. While using ac they make climate change even worse my devils circle but those idiots won’t understand. Wouldn’t mind they be cooking with 50+ degrees', "That would be a scary sight! Cows and panels all over everywhere.  So maybe don't put the solar farm in known tornado areas.  ( southern alberta where im from) is full of large grass lands no trees and mostly cattle ranches.  With climate change we are going to see more extreme heat events like this one.   This could be a solution for places without tornadoes maybe ???", 'I don’t think that anyone is saying that carbon emissions are the sole driver of climate change but it’s one of the largest ones. It’s also one area we can easily fix if not for politics/lobbying by major fossil fuel industries which makes it frustrating to talk about. \n\nYou’re right that a weakening magnetic field does contribute but the best guess we have as to why it’s weakening is because of something happening some 4,000 mi below the Earth’s crust that we can’t really fix aside from using some kind of space-age technology. It seems like we’re also not entirely sure of the exact rate it’s decreasing or how the current intensity has dropped in the past 200 years (I’ve seen estimates from 8-10%) so there’s some estimation going on as to how much it contributes to the issue (which again, no one is saying is 0%).', 'Wicked definitely felt dated to me as well.  It has this mentality that "bad things are happening because there are bad people running the government, and once we get rid of them things will get better." \n\nTo me this is a complete relic of the Bush years.  Since then we\'ve, sadly, discovered that things don\'t just get better when the wizard is gone, because people wrap their entire identity into the political party they support, so will never abandon it.  I\'d say Hadestown much better fits today\'s political view since we know from the start that it\'s a tragedy, but "maybe it\'ll turn out this time" which of course it doesn\'t.  That\'s about how I feel about climate change, American democracy, resource depletion, and the survival of our civilization.', 'At last. Lismore is a failed settlement. 30 Major Floods(&gt;9.7 metres) in last 150 years and another 10 that fall just short of Lismore Council\'s definition of a Major Flood. It\'s like watching villagers flee from villages built at the foot of volcanoes and you ask \'Who lives next to a volcano?" Before anyone gives me the \'only the poor live there because of housing affordability\' argument consider that there are numerous inland towns in NSW that are crying out for inhabitants and offering all sorts of housing concession deals. People in Lismore choose to live in Lismore and surrounds. They like the riparian aesthetic. It\'s the same as those who build on ridgelines next to National Parks. They are there for the aesthetic. The location they choose to live in is, has been and will always be hazardous and life threatening..not just to them but their rescuers as well.\n\nOf course there will be redditors on here that decry my stance but repeatedly(and rightly so) demand governments to do something about climate change yet at the same time want to prop up settlements that will be smashed again and again by the effects of that climate change. Hypocrites.\n\nI\'ve voted Labor for 51 years and I\'m a pragmatist at heart. I would rather see my tax dollars used to relocate the people of Lismore to inland cities/towns (and I know some of them suffer from flood, fire and drought..it is Australia..but not on average of every 5 years or less) than see it literally washed down the drain by hanging on to an emotional but totally unrealistic idea of rebuilding.', 'I talked about stuff like this with my SO recently. The world is going to shit, and its doing so *fast*. More and more nature catastrophies (NSW fire, massive NA fires, followed by mass flooding all over the world etc.) In addition the global swing towards right wing authoritatianism, and wars. The climate goals are a joke as even if wed reach them, itd be faaaar too late. This planet is fucked... personally I see 3 or 4 future scenarios: 1. Natural desasters and climate change kills majority if not all of humanity and life on this planet, 2. Wars will do it. 3. Some of us manage to escape to space and build a new society. Im not looking forward toqards the future... everybody is saying that I need to save money so that when Im retired I can live happily. First: even if I ssave money and reach retirement, when politics continue like they are, that money will be worth fuckall by that time. Second: Im 25... I doubt Ill ever reach the age of retirement... and if I somehow do, the world and society will look veeeery different.... so I do save some money for emergencies etc. But I also spend a fair bit because I might as well enjoy the world as it is right now as much as I can before it reall, goes to shit...', "IS NOWHERE SAFE?! \n\nI realize there are far greater ramifications of climate change but WON'T SOMEONE THINK OF THE PREGNANT?!", 'The article says "clean coal" is a technology brand created by the coal industry, named as such because it captures more coal waste (coal burning is not clean) than older power plants. And 10C higher is world-changing. Today\'s climate change is primarily anthropogenic in its nature and cause, driven by technology and processes that were not around 5 million years ago. Thanks, but there\'s no need for us to have a longer discussion about this after you get home.', "The worse climate change gets the more diseases we are going to see.  Covid is just the first pandemic that we are seriously experiencing on an entirely world wide scale.  Developed countries have mostly avoided pandemics in the past.  Covid is showing us that we won't be able to avoid them entirely.\n\nA starving person is a desperate person.  As traditional food sources dry up people will naturally search for alternate food sources.  These alternate food sources will expose us to diseases that we don't have immunities to.  Cheap accessible world wide travel makes it easy for these diseases to spread before they are noticed.\n\nWe can't continue the way we have been.  We have options, but as things get worse our options are being reduced.  We must change.  We can't just blame the politicians.  They will do what they must to get reelected.  We must change what we consider important.  We create the political will.  The power is ours.  We affect businesses with how we choose to spend our money.  We affect politicians with how we choose to spend our votes.\n\nIf you live in a democracy the power is yours.  Many have been made politically apathetic.  That is what people who want to maintain the status quo want.  They don't want the masses to realize the power is theirs.\n\nThe power to ensure our survival is ours.  We need only to reach out and grasp it.", 'How about instead of making assumptions about my knowledge of veganism you refute one of my arguments with you know... an actual argument of your own? You think you are on a high horse cause of your fucking diet. Whatever. Lol if you wanna change the world then vote and push your local and federal government to elect people who care about climate change. Industry is evil but it doesn’t change the way you think it does. It changes through government.', '&gt; Faker has been downloaded 121M times. 121M x 3.3 MB = way too many Bytes.\n\nThat\'s stupid... that\'s like saying "Google\'s homepage has been download 10000M times, 10000M x 1MB = way too many Bytes".\n\n&gt; I\'m super concerned about the responsibility that we developers have on climate change. Over the years, Faker has probably emitted more than 11 Metric tons of CO2 equivalent.\n\nAgain stupid, see previous argument. Everything does this. Plus I\'m highly sceptical of the science behind this "probably" estimate.\n\n&gt; Most people only use 1 locale, so they only need a fraction of the library size. Yet, because it\'s designed as a multilingual library, there is no alternative:\n\nYes, discourage accessibility to minorities... great move.\n\n&gt; they have to download the 3MB even if they only need 10KB.\n\nBoo-frickin-hoo... oversized files are everywhere.\n\nI got no time to go through this whole article but a lot of the issues seem exaggerated to say the least, and until there is an alternative or attempt to fix so-called issues, people will just keep using it anyway. I\'m not sure how not continuing to work on it and attempt to solve these "issues" is supposed to help anything.', "Honestly the Mad Max wasteland at least as a setting is probably pretty accurate to how things would be after severe climate change and nuclear war. People looking for scraps of food anywhere, huddling around the few sources of clean water left. People dying young due to cancer as it ravages populations unchecked.  \n\n\nI know a lot of critics like mentioning how society in Fury Road seems to of collapsed generations ago but in reality if you take into account that central government control was already near to falling apart in the first Mad Max then put severe climate change, on top of nuclear war on a already oil depressed world things would just domino.\n\nNot to mention people dying young, so parental care and even education having trouble reaching the kids who've been born into this hellscape. Not to mention that in this world how much of that pre-apocalypse knowledge would be considered useless.   \n\n\nThese days too many settings seem REALLY eager to try and portray some sort of colourful wasteland. Where there is always hope to be found and even if things have been destroyed it's not all that bad.   \n\n\nMad Max puts forward the idea that maybe hope is a bit harder to find than we think, maybe humanity isn't so resilient that we don't regress when all our progress is taken away but I like that the message also seems to be that sometimes a few acts of selflessness regardless of motivation can make a difference.  \n\n\nCurrently playing through the game, I think in some too  many ways it leans into the Apocalypse happening almost generations ago meanwhile mentioning how it was recent. though main issue with the game is how most of the main characters are voiced by Americans yet by making it based around Gastown it's still based in Australia.", "&gt;Well, in Australia abortion law is up to the states. It's not the US\n\nUp until 1973, it was up to the states in the US as well. And then the supreme court realised controlling women's bodies was illegal and lealised it across.\n\n&amp;#x200B;\n\nI am aussie. The current government is as right wing as the GOP in many cases and more so in others (like climate change).\n\nThe current government in Aus literally does not believe climate change is a real thing.....\n\n&gt;Currently all states except South Australia allow abortion. A referendum would probably pass.\n\nWrong, it's restictive in others too. For example even in NSW an SA resident can't visit to get an abortion.\n\nIt's abhorrent seeing women's right protests after Trump was elected when women don't have rights to their bodies in many parts of Australia.\n\n&gt;  But they're not far-right. \n\nEvery policy i states is a far-right policy. Youre using words but your words dont have substance.", "Yeah but ending fracking won't reverse climate change anyway, you need also need many many other industries to change for that to happen. Also I'm talking about Biden's policies, not about climate change. I'm not really sure why you commented as you did.", '&gt; Michael Johnston was driving home to Round Rock from a protest in Austin in 2016 when he saw blue lights flashing on the side of the road. He had attended an event with the Peaceful Streets Project that night, during which people filmed police officers to increase accountability, and he said that was the main reason he pulled into a parking lot across from the police car.\n&gt; \n&gt; What followed was an interaction Johnston describes as uneventful — a brief conversation with two Round Rock officers before he drove away. The police report filed after the incident says that the officers were dealing with an intoxicated man, and that Johnston told them he had pulled over the make a phone call. The report mentions that Johnston was wearing a Peaceful Streets Project T-shirt and wristband, would only roll down the windows part way and filmed the interaction on his cellphone.\n&gt; \n&gt; It wasn’t until May 2019, when Johnston called the police after his vehicle was rear ended, that an officer told him that the Round Rock Police Department had an alert on his name, labeling him as "not police friendly."\n&gt; \n&gt; Johnston said he does not have any other criminal record and has never been arrested.\n&gt; \n&gt; The department like many across the country, including Austin’s uses alert systems to share information among officers.\n&gt; \n&gt; Police say such alerts make both officers and the public safer by providing officers with necessary information. However, national experts acknowledge that such systems could influence officers, and statewide law enforcement advocates say alerts should only be applied when someone’s behavior puts officers or the public at risk.\n&gt; \n&gt; Round Rock’s system has flags for people with mental illness, communicable diseases or a history of violent interactions with law enforcement, among others, Police Chief Allen Banks said.\n&gt; \n&gt; Critics say that because officers can apply alerts without oversight, the alerts could be inconsistent and might cause unnecessary tension when officers interact with those individuals.\n&gt; \n&gt; Amid the recent protests against police brutality, Johnston said he decided to raise his concerns at a Round Rock City Council meeting in early July.\n&gt; \n&gt; "Given the climate in the country now, trying to make even a slight change in police policies that I don’t agree with seems like the right thing to do," he said in an interview with the Statesman.\n&gt; \n&gt; Johnston told council members that he wondered what a police officer would think of his own profile: a veteran who at one point had a license to carry a handgun and was labeled as "not police friendly."\n&gt; \n&gt; "Police officers are human, they’re fallible and, if they feel like the person they are talking to is not friendly in a way that is antagonistic, I don’t believe that situations are going to be less likely to be escalated," he said. "I think there’s a higher potential for a bad outcome."\n&gt; \n&gt; Banks disagrees with Johnston’s assessment. He said the alerts are an important tool and are matter of public and officer safety.\n&gt; \n&gt; "It doesn’t escalate a situation. What it does is allows an officer to prepare for any engagement or interaction," he said. "We have this set up as a safeguard in our community."\n&gt; \n&gt; Alerts can be placed on a person or address at the discretion of an officer and can be removed later if another officer feels the alert no longer applies, Banks said. An alert might be removed if someone who has been deemed "not police friendly" has a subsequent positive interaction with officers or if the department finds a flagged address has new occupants.\n&gt; \n&gt; Banks said it is hard to quantify how many alerts exist in their system at any given time.\n&gt; \n&gt; The Round Rock department has no plans to change the system. He said that since Johnston raised the issue to the City Council, the department has had several positive interactions with him and his alert was removed.\n&gt; \n&gt; While the Round Rock Police Department does not share its alerts with other law enforcement agencies, other departments use similar systems.\n&gt; \n&gt; Austin police Lt. Kenneth Murphy, who manages the Emergency Communications Division, said his department uses caution notes for addresses that are created through a computer-aided dispatch system.\n&gt; \n&gt; Officers often receive information about caution notes — such as warnings that someone is anti-police, known to have weapons or has a dangerous dog — from dispatchers while responding to a call, Murphy said.\n&gt; \n&gt; Brandon Gilstrap, the department’s central records manager, said there are many person-based alerts, including anti-police, career criminal, dangerous, mentally unstable, and gang member, as well as alerts for people who are known to use narcotics or carry weapons.\n&gt; \n&gt; Murphy said the notes help officers handle encounters effectively, such as employing de-escalation techniques or bringing an officer with special mental health training.\n&gt; \n&gt; "Force escalates when officers are caught off guard," he said. "If officers are prepared, then they’re more likely not to have issues and it’s safer for the citizens."\n&gt; \n&gt; Gilstrap said the Austin Police Department shares its alerts with certain agencies and offices, such as the fire marshal and district attorney.\n&gt; \n&gt; Jennifer Szimanski, the public affairs coordinator for the Combined Law Enforcement Associations of Texas, said most large law enforcement agencies in Texas use an alert system. However, she said those alerts should be used for concrete public safety concerns and that her organization would not support agencies using designations like "not police friendly" if they are not based on an interaction such as an assault on an officer.\n&gt; \n&gt; "It’s not supposed to be used for any type of opinion," Szimanski said. "If you interact with someone who isn’t friendly with you or was maybe annoyed or pissed off that you were there, it wouldn’t be something that you would put in the notes. It would have to be something that occurred with that person or that resident that would affect public safety and officer safety."\n&gt; \n&gt; Szimanski said it can be difficult to know from the outside how an officer or department is using any given alert since the language involved is not standardized. In her experience, departments often use the "anti-police" and the "violent against police" alerts interchangeably. She said that her organization would support the standardization of alerts for clarity, as long as officers are still able to access information about prior incidents before responding to a call.\n&gt; \n&gt; Maria Haberfeld, a professor of police science at the John Jay College of Criminal Justice in New York, said the alerts are an effective way for officers and departments to identify environments and individuals that are responsible for criminal activity or threats against the community.\n&gt; \n&gt; However, the system can cause officers to arrive with preconceived notions of what they will find, she said.\n&gt; \n&gt; "If somebody’s known for having a violent interaction with the police before, they will arrive preparing for the worst. And when you arrive preparing for the worst, you arrive with a heightened state of stress and anxiety," she said. "This might precipitate the use of force."\n&gt; \n&gt; There is no way to completely eradicate this negative effect, Haberfeld said.\n&gt; \n&gt; "The way we process knowledge is directly related to our emotional maturity, emotional intelligence and social intelligence, and unfortunately we don’t necessarily recruit people based on this standard," she said. "I am afraid we are not going to be able to anymore, given the current anti-police sentiment."\n&gt; \n&gt; Overall, Haberfeld said she believes the alerts are a necessary part of police work.\n&gt; \n&gt; "There is no way that police officers can just turn off the switch related to prior knowledge. It would be dangerous for them and dangerous for the public," she said. "The alerts are something that’s in the interest of the public, but there’s always this underlying query of: safe at what price? It’s for everyone to answer individually. It’s certainly not designed to oppress or profile anybody."\n&gt; \n&gt; With these alert systems in place, Johnston said he wants other people to know they exist. He said that if he was not a "fairly non-threatening" white man, police might not have volunteered information about the alert system to him in the first place.\n&gt; \n&gt; "I do think it’s important for average citizens to understand that these systems are out there," he said.', "&gt; Based on the history of France, who converted half their grid to nuclear in just 15 years, and the recent history of Germany, who has spent comparable time and money on renewables while getting almost nowhere, we can do much faster with nuclear. The idea of waiting on this - IMAO pipedream - hope of much much cheaper batteries is the most irresponsible thing that I have ever heard because of the immediacy and severity of climate change and ocean acidification.\n\nWell, if you're hoping on America adopting nuclear energy as the solution, I have some real bad news for you:  It ain't happening.  \n\nThe last nuclear reactor built in the US took 17 years to build, 23 years to actually turn on.  \n\nIn the meantime, there's been an unofficial phase-out of nuclear reactors in the US, because they take forever to build, they're stupidly expensive and they take forever to see a return on the investment.  \n\nThat's just the reality of nuclear power in the US.  This is why renewable sources are so much more popular.  To say nothing of not having to deal with the stigma of being nuclear.  I mean, the dangers of nuclear reactors are essentially a part of American culture.  And there's no getting away from that.  Nuclear, as a broad-based go-to power source, is dead.  Now, we'll see it continue as a niche power source for things like space travel and military vessels, but that's about it.", 'Saying something is true doesn’t mean it is. You only provided one argument this entire time with that being that since many Americans think the news is mainly left biased then it must be which is the dumbest argument ever. Every other source you provided scored worse than Fox News on the bias and credibility scale so you can’t honestly tell me the source is trustworthy in any way shape or form. Like I said before, you likely wouldn’t take a CNN article I showed you into consideration and why should you? The site is untrustworthy and biased but for some reason you think that the source you provided (that actively supports conspiracy theories and climate change denial) has any sort of credibility let alone enough to win an argument by itself.', "&gt;No, it's not. There is a lot more to communism. Communism is actually impossible without global co-operation so I pretty much switch off when anyone claims themselves, or others, to be communist. This is socialist.\n\nDefine communism then.\n\n&gt;I asked if Sanders could be considered far left. Public ownership of certain companies or industries is not uncommon in western democracies, and is often supported by centrist parties. So I really don't believe this alone makes Sanders far left.\n\nNot a few companies, multiple major industries. There is no way you don't understand the difference. If you keep pretending that you do not understand it I'll have to assume you're not arguing in good faith and I'll end the discussion.\n\n&gt;Sorry, but I find this incredibly weak. \n\nIf a politician expressed radical opinions in the past and refuses to correct them when asked about them I am going to assume he still believes them.\n\n&gt;He also wants the federal government to prevent people coming to US for healthcare.\n\nHe wants to decriminalize illegal border crossings. Please explain how he is going to prevent illegal aliens from hopping the fence and running to the nearest hospital when they have a medical issue they need taken care of.\n\n&gt;Almost all late term abortions are performed only in medical emergencies.\n\nCan you find a single source that claim that doesn't include mental health? Because I haven't found one. Also, which medical emergencies would necessitate a late term abortion? \n\n&gt;Again, in the 70s.\n\nAgain, never explained why or even admitted that it is wrong.\n\n&gt;Yeah. Action on climate change, higher taxes, removing legalised political bribes, universal healthcare, living minimum wage, legalised abortion. All these things are widely supported by most centre-left governments in the developed world.\n\nNo, they are not. There isn't a single socialist party in the West that wouldn't welcome Sanders with open arms. \n\nYou are seriously arguing that taxpayer funded late term abortions and outlawing private health insurance are centrist opinions. You are either completely disingenuous or you have no clue what you are talking about. Probably both.\n\n&gt;I honestly just don't see this as far left. \n\nYour radicalism doesn't make an open socialist a centrist.\n\n&gt;To me, far left is a gutting of capitalist institutions to be replaced by socialist ones, with the goal to move ALL means of production to the working class.\n\nThis is like an anarcho-capitalist saying a politician isn't far right unless they oppose the very existence of government and that libertarians like Rand Paul are actually just centrist.", 'Thanks for your response. I would love to have a discussion. You imply some arguments against what ive written and i would really like to respond to them if you would give me some sources so that we are on the same level of information. \n\nI need the following sources: \n\nSource that evidently states that the scientific consensus is build through an "online quiz".\n\nSource about data contradicting manmade climate change.\n\nSource about China lying about their energy policies, especially as it is stated in the sources of [this](https://en.wikipedia.org/wiki/Energy_policy_of_China) wiki article.\n\nAnd then i want a source in which I blame the west or condemn nuclear energy, because i think you are implying i do but im not.\n\n&amp;#x200B;\n\nThank you.', 'Well kinda. Conditions are horrible because of climate change but the fires themselves needed for want of a better word a spark to ignite them. It quite possibly was down to both. Humans started the fire and the fires are worse than they would normally be due to climate change.', 'Want to combat climate change? Move out of the city and be self sustainable.\n\nEvery person that lives in and moves to this city is guilty of consuming, over production, and more money being budget for more services.\n\nYou want less production but live in a city that requires 24 hour consumption.', 'The fact that they never even bothered to use actual drinking glasses, plates or silverware has driven me up the wall for years. So much plastic and styrofoam waste, it was ridiculous. I mean sure they have an enormous family to feed but you already have that restaurant size kitchen- you’re telling me you didn’t bother to add the restaurant sized dishwasher?? \n\nI remember one episode in which the kids had a stomach bug and they had them carrying around those red plastic cups to vomit into. Why? Are they not allowed to vomit into the toilet? So many questions. And yeah, I remember the fan Q&amp;A show in which they stated that the earth being overpopulated is a myth and there’s actually plenty of room. Yikes. \n\nBut really, it’s all a moot point. They don’t believe in climate change because God will take care of everything and it’s all his plan and Jesus and blah blah blah something transgendered persons are evil. For all the world travel they’ve had the opportunity to do they are incredibly ignorant.', "One of many reasons I have drifted away from the Conservatives. Me in a snapshot:\n\n- rural small town white guy (Manitoba)\n- political DNA in rural Canada is Conservative by default\n- love politics and in my younger years volunteered for Conservatives through most of Harper era\n- I happen to be gay but have generally in the past felt ok in the Tories as they're is *so many gay Conservatives* and Baird did a great job standing up for global LGBTQ rights and to the party's credit they amended their consistutuon to remove the anti marriage equality plank\n\nHowever, in the lead up to writ dropping for the 2015 election I cancelled my Conservative membership and voted Liberal for Trudeau. Seeing Harper do the barbaric practices line, campaign with Rob Ford and just get *so desperate* was my point of not return.\n\nI had always had many areas of discomfort. I'm a British Columbian and see the carbon tax as the fiscal conservative solution to climate change. All to rally up their base they throw out principles for populism and go on this anti carbon tax stance.\n\nCouple that with long form census, and other policies that were ideologically driven away from data and I was getting more uncomfortable. Still, I felt my more socially progressive / centre-to-centre-right fiscal orientation had a home still in the Tories with people like Michael Chong etc.\n\nTo me conservatism was supposed to be about individual liberty, market based policy solutions, fiscal responsibility (boutique tax credits, cutting gst when consumption taxes are economically sound and income tax cuts are what make most sense pissed me off too).\n\nScheer has cemented me as a centre right Liberal. Unequivocally, from the SoCon wing of the party and the Canada food guide bullshit and this pathetic omission continuing with still no Tory leader in history to ever March in a pride parade make it clear to me the party doesn't value you me at all.\n\nYes, the folks will come in here saying it's vote buying in a parade and virtue signalling and perhaps, but to me being a kid who was so damn alone in my white conservative small Prairie town being gay it fucking matters.\n\nIt means you see my humanity. And when the Tory leader always goes to every major ethnic event under the sun (Chinese New Year parade, Vahsakhi etc) they damn well should prioritize this in their calendar.\n\nI have many, many critiques of the Liberals and Trudeau but at the very least I feel my humanity is recognized and I have a true champion for the equality of my community. Being more fiscally to the right I feel I have a place in the Liberals and not this slow feeling of seeing my former party leave me.\n\nNever say never with a future iteration of the Tories I could vote for them again but this is why I see it still as Trudeau's election to lose...\n\nThis iteration of the Tories is too pandering to social conservatism for urban moderates to want to consider them and we're essential swing voters they need.", "Global warming is correct, its name just switched to climate change so that idiots in select parts of the world didn't hold up a snowball and say it didn't exist. \n\nThats not a study. It references a study, but does not actually show any study. The site the news is from is taken from newsbusters, which is not a neutral source. I assume you know that you didn't link a study and are trying to fool me. I couldn't even find the names of the scientists it referenced for the study in my quick read through of it, so I wasn't able to find the study myself.\n\nI did a bit of research on my own, and at least according to wikipedia, of the people who have some beef with the current assertions of climate change, only 4 people believe that it will not have little negative impact. Literally everyone else agrees that there will be a decent amount of negative impacts, but disagrees about how much the climate is changing. Also, in that list there is not a single pure climatologist who thinks that.\n\nhttps://en.wikipedia.org/wiki/List_of_scientists_who_disagree_with_the_scientific_consensus_on_global_warming\n\n^ Link for reference. If you find others I will be happy to give them a look up.\n\nDid you look at the temperature trend? Do you think its normal the rate its increasing?\n\nAnd ignoring that, what do you think will happen with climate change? I assume you think something will happen, because the ice caps will melt at the very least (and we can clearly SEE them melting).", "&gt;I sadly believe the thing has become political within the science community. Scientist who have published results going against the narrative have lost funding. That’s the last thing they want so many just follow along.\n\nDo you have any evidence for such a large scale conspiracy? There would have to be massive coordination across the the entire globe to pull off a conspiracy of that magnitude. What you're alleging is different than misaligned incentives. If you think that the entire field of climate science is completely corrupt to the point where they're making all of the science up in regards to CO2 being a primary driver of climate change and humans being responsible for all or most of the warming in the last half a century you would need some pretty strong evidence to corroborate such a claim.\n\n&gt;I think it has turn to political and believe it will hurt poorer countries more than developed ones.\n\nWhere's your evidence for this?\n\n&gt;Ice core samples have shown less cO2 in our atmosphere in the past around the same time Scotland had vineyards and Europe was going through a long warm spell.\n\nCan you give me a source for that? Here's an article that gives some thoughts on the 'vineyards in England' argument: [http://www.realclimate.org/index.php/archives/2006/07/medieval-warmth-and-english-wine/](http://www.realclimate.org/index.php/archives/2006/07/medieval-warmth-and-english-wine/)\n\n&gt;Then ice core samples have shown large amounts of C02 in our atmosphere during times of long periods of cold.\n\nCan you show me which period in Earth's history you're referring to? The general response to this would be that there isn't a 1:1 correlation with CO2 in the atmosphere and temperature. The climate is a complex system and solar forcings and other variable all contribute. The reason we've been able to isolate human-caused GHG's as a main driver today, is that we understand the other variables well enough to show that they can't be responsible for all of the warming we're seeing.\n\n&gt;C02 is a very important for life on Earth obviously but times when there was more in our atmosphere everything was fine.\n\nOf course, which is why all scientists say that it's not the raw number of CO2 in the atmosphere that matters but the rate of change. Current lifeforms, including humans, have evolved to thrive in certain climate conditions... for instance, there are many studies being done on how climate change is impacting insects. As you noted, CO2 can have the effect of increasing the biomass of plants... however, this increase in many plants has meant a lower nutrient to biomass ratio... essentially the plant grows bigger faster but has less protein. Certain insects have evolved to regulate their food intake based on biomass, not amount of protein. So they will eat what they have evolved perceive as adequate amounts of food but the lack of nutrients will leave them more vulnerable during the winter and to disease. A similar phenomenon is happening with nutrients in food grown for humans. So it's not as simple as saying CO2 is essential for life, therefore any rate of change in atmospheric CO2 is good. I don't know why we wouldn't expect to see problems when we're essentially conducting a science experiment on the earth by releasing massive amounts of carbon that had been stored underground. There's no free lunch on earth... there will be costs for such cheap sources of energy.", "It's different when you can individually have an effect on it. Obviously it's somewhat of a waste of energy to worry about things you can't do anything about, but climate change is not one of those things.", "&gt;No one is going to care how cheap your energy was when climate change has killed you and everyone on earth.\n\nThat's why we urgently need to decarbonise. The current halfway house we are heading for, a mix of wind/solar and natural gas, is completely insufficient.\n\nThe Energy Research Partnership has calculated that wind power can reduce UK emissions to 180 grams of CO2 per KWH. That's far beyond the target we need to meet. They also calculate we can reach 50 grams with an expansion of nuclear power.\n\nHave a look at: https://www.electricitymap.org/?wind=false&amp;solar=false&amp;page=country&amp;remote=true&amp;countryCode=FR\n\nWind speeds are quite high at the moment. Germany and Denmark, despite the vast amounts of wind power they have installed, are still emitting very large amounts of CO2. Areas that are meeting the emissions targets use either hydro or nuclear, or a combination of both.\n\nSadly the debate in the west has been hijacked by renewables advocates who come up with ridiculous ideas like turning renewable electricity into hydrogen to replace gas, even though it's far too expensive to be practical. Germany has spent hundreds of billions on renewables, but they haven't replaced their coal and gas plants, because intermittent wind and solar require firm backup. We need to face reality on the environment. We need to cut carbon emissions, but using wind and solar, which only work some of the time, isn't going to be enough. ", "Becuase slavery and modern working conditions are SO similar? \n\nClimate change is a major concern to most people I agree  but the drastic change required to fight climate change takes time. Take France for example, they are phasing out petrol and diesel cars by 2040. Something like that doesn't happen over night.\n", "Usually Climate change denier is used to reference someone who doesn't believe humans are at least part responsible for accelerating climate change with pollution of some sort. They usually are against putting possible solutions in place as they dont think our carbon footprint is doing anything.\n\nThat part is also fact and anyone denying that is also scientifically inaccurate. (they are also, interestingly enough, usually attached to a oil company through so many links)\n\nyes the glaciers melted, but it has nothing to do with the current conversation about climate change. The glaciers melted because they were going to anyway. The current problem we are facing is man made.", "Dude, I am so sorry. Check my posts, I am a landlord as well as a teacher/IT person. Fleas and bedbugs are absolutely miserable to get rid of. \n\nTo echo the other landlord here (what up! Haha)\n\n1. Always wear pants and long sleeves. \n2. Their eggs live in fabric easily. Make sure you change outside your house or in your garage when you come home. Don't bring them in your house, take them to the laundromat. You need to keep your spouse/pets/house flea free while your school's facilities team deals with them.\n2a. Get 2x 5 gallon ziploc bag, take off your shoes and socks before you go into your house. Put the socks with your work clothes, and the shoes in the 5 gallon ziploc. Wear a different pair of shoes the next day and let the first pair bake in the sun outside your house, it should get hot enough inside the bag to kill any fleas and eggs if it has prolonged direct sunlight. Then alternate with your other pair of shoes. If in a colder climate, get a cooler and freeze that shit to produce the same effect.\n3. Try and remove as much fabric from your room as possible, rugs, unlaminated wall hangings, etc. ask if they can remove any carpets from your room, or give you one of the clear chairmat things that cover them temporarily. \n4. If you have any pets, make sure they have a flea collar. Frontline is bullshit (ok, its not, but its nowhere near as good as an actual collar) and change it every month or so. \n5. Hope that they don't get into your house. Seriously, fleas are the absolute bane of my landlord existence. ", "The fact that the storm is being fed by a super hot Gulf of Mexico and bumping into two other fronts being pushed by warm air from the south.  Trying to pretend that the weather pattern isn't in any way effected by the climate is like saying your high blood pressure isn't about what you eat or how much you exercise.   Now to extend that metaphor, the time to discuss diet and exercise isn't after you've had a heart attack so in that respect, talking about climate change now is a bit day late dollar short.   While I strongly agree that climate change is an issue at hand here, I don't think anyone is houston really wants to hear that right now. \n\nBut let's be real.  Of course the frequency of these storms and the absolute deluge of these rain events (Baton Rouge last year, Meyer land last year etc) are results at least in part of climate change.  When you're having a 100 year flood every year or so it's time to consider that perhaps the earth is just different now.  \n\n\nEdit: And just a note- there is no logical reason for climate science to be a political divide.  There is nothing about being a conservative that should make someone unable to understand facts, trends and predictions based on scientific models.   This tribalism has got to stop.   Why on earth is it even a discussion anymore.  The only people on the planet still denying climate change are this subset of republicans.   It's makes no sense at all.  ", "His stance on climate change for the interest of business could turn around and bite him in the ass. If there is one thing that should hold factual ground it should be science and especially scientific fact, pretending the science isn't true could probably be used in this manner."]


Overwriting test_data.py


In [ ]:
!pytest . -vv

Test session starts (platform: linux, Python 3.7.14, pytest 3.6.4, pytest-sugar 0.9.5)
cachedir: .pytest_cache
rootdir: /content, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_data.py::test_without_inferior_limit ✓                      33% ███▍      
 test_data.py::test_without_superior_limit ✓                      67% ██████▋   
 test_data.py::test_with_both_limits ✓                           100% ██████████

Results (139.06s):
       3 passed


### Function 3

Given a parameter value, return two messages whose sum of the value of the "score" column is equal to the parameter. Return -1 if it doesn't exist.

In [ ]:
class Climate():   
  def __init__(self, csv_filename):
        with open(csv_filename) as f: 
          reader = csv.reader(f)
          rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]

        for row in self.rows:              
            row[-1] = int(row[-1])

        self.scores = {}                    
        for row in self.rows:                       
            self.scores[row[-1]] = row

  def check_score(self, target):    
      '''
      Description: 
      This function checks if there are or not two values 
      from the last column of the array whose sum is equal 
      to the target value.

      Parameter:
      - target <int>: integer that represents the target value

      Return:
      Two messages if sum is equal to target 
      or -1 if no message is found
      '''
      for row1 in self.rows:                  
          for row2 in self.rows:
              if row1[-1] + row2[-1] == target:
                  return row1, row2
      return -1     

  def check_score_fast(self, target):
      '''
      Description: 
      This function checks if there are or not two values 
      in the dictionary named 'score' whose sum is equal to 
      the target value.

      Parameter:
      - target <int>: integer that represents the target value

      Return:
      Two messages if sum is equal to target 
      or -1 if no message is found
      '''
      for row in self.scores:      
          row_target = target - row         
          if row_target in self.scores:
              return self.scores[row], self.scores[row_target]
      return -1          

In [ ]:
climate = Climate(path) 

In [ ]:
print('1:', climate.check_score(40))
print('2:', climate.check_score_fast(40))

1: (['comment', 'imlddn9', '2qh3l', 'news', 'false', '1661990368', 'https://old.reddit.com/r/news/comments/x2cszk/us_life_expectancy_down_for_secondstraight_year/imlddn9/', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', '0.5719', 2], ['comment', 'imk0keh', '3l2gt', 'walkaway', 'false', '1661970972', 'https://old.reddit.com/r/walkaway/comments/x2gmkp/how_to_charge_your_electric_vehicle_in_california/imk0keh/', 'Its not a mistake. Theyre wrecking everything under the guise of "climate change".', '0.2584', 38])
2: (['comment', 'c0i28hb', '2qh1n', 'environment', 'false', '1262383373', 'https://old.reddit.com/r/environment/comments/akl0m/is_the_airborne_fraction_of_anthropogenic_co2/c0i28hb/', 'No one truly knows what\'s going on,

In [ ]:
print('1:', climate.check_score(0))
print('2:', climate.check_score_fast(0))

1: (['comment', 'imlddn9', '2qh3l', 'news', 'false', '1661990368', 'https://old.reddit.com/r/news/comments/x2cszk/us_life_expectancy_down_for_secondstraight_year/imlddn9/', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', '0.5719', 2], ['comment', 'imkvel6', '2qh6c', 'christianity', 'false', '1661982640', 'https://old.reddit.com/r/Christianity/comments/x2d3g0/major_evangelical_organization_says_christianity/imkvel6/', "You are correct. I have no concern for climate change. I leave that in God's hands to not man's.\n\nHow would you express Christ to the world?", '-0.34', -2])
2: (['comment', 'c0i28hb', '2qh1n', 'environment', 'false', '1262383373', 'https://old.reddit.com/r/environment/comments/akl0m/is_the_airborne_fraction_of_

In [ ]:
print('1:', climate.check_score(500))
print('2:', climate.check_score_fast(500))

1: (['comment', 'imlddn9', '2qh3l', 'news', 'false', '1661990368', 'https://old.reddit.com/r/news/comments/x2cszk/us_life_expectancy_down_for_secondstraight_year/imlddn9/', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', '0.5719', 2], ['comment', 'im9i19f', '2qh3l', 'news', 'false', '1661785239', 'https://old.reddit.com/r/news/comments/x0fabf/last_member_of_indigenous_tribe_dies_in_brazil/im9i19f/', 'It’s literally the one reason we will wipe ourselves out via climate change. We want money. It’s that simple. And no local government on earth will chose long term survival over corporate profits.', '0.25', 498])
2: (['comment', 'c0i28hb', '2qh1n', 'environment', 'false', '1262383373', 'https://old.reddit.com/r/environment/comment

#### Comparing performance

To compare the performance of the `check_score` and `check_score_fast` functions, a list with random values was created.

In [ ]:
scores = [random.randint(0, 200) for _ in range(10000)]

total_time_no_dict = 0                                    
for score in scores:                                     
    start = time.time()                                  
    climate.check_score(score)             
    end = time.time()                                   
    total_time_no_dict += end - start                    
    
total_time_dict = 0                                       
for score in scores:                                    
    start = time.time()                                 
    climate.check_score_fast(score)       
    end = time.time()                                   
    total_time_dict += end - start                        
    
print('Time (without dictionary): ', total_time_no_dict)                                
print('Time (with dictionary): ', total_time_dict)

Time (without dictionary):  9.010575532913208
Time (with dictionary):  0.0057277679443359375


As we can see, the function `check_score_fast` (implemented with a dictonary) was much faster than the function `check_score` (implemented with `for`), in fact, it was 9.01057/0.005727 = **1573** times faster.

#### Test

In [ ]:
%%file test_data.py
import pytest, csv

class Climate():   
  def __init__(self, csv_filename):
        with open(csv_filename) as f: 
          reader = csv.reader(f)
          rows = list(reader)
        self.header = rows[0]         
        self.rows = rows[1:]

        for row in self.rows:              
            row[-1] = int(row[-1])

        self.scores = {}                    
        for row in self.rows:                       
            self.scores[row[-1]] = row

  def check_score(self, target):    
      '''
      Description: 
      This function checks if there are or not two values 
      from the last column of the array whose sum is equal 
      to the target value.

      Parameter:
      - target <int>: integer that represents the target value

      Return:
      Two messages if sum is equal to target 
      or -1 if no message is found
      '''
      for row1 in self.rows:                  
          for row2 in self.rows:
              if row1[-1] + row2[-1] == target:
                  return row1, row2
      return -1     

  def check_score_fast(self, target):
      '''
      Description: 
      This function checks if there are or not two values 
      from the dictionary called scores whose sum is equal 
      to the target value.

      Parameter:
      - target <int>: integer that represents the target value

      Return:
      Two messages if sum is equal to target 
      or -1 if no message is found
      '''
      for row in self.scores:      
          row_target = target - row         
          if row_target in self.scores:
              return self.scores[row], self.scores[row_target]
      return -1                      

@pytest.fixture(scope="session")
def data():
    climate = Climate('/content/drive/MyDrive/dataset/the-reddit-climate-change-dataset-comments.csv')
    return climate

def test_1_check_score_two_msg_1(data):
    """
    Test 1
    """
    assert data.check_score(40) == (['comment', 'imlddn9', '2qh3l', 'news', 'false', '1661990368', 'https://old.reddit.com/r/news/comments/x2cszk/us_life_expectancy_down_for_secondstraight_year/imlddn9/', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', '0.5719', 2], 
                                    ['comment', 'imk0keh', '3l2gt', 'walkaway', 'false', '1661970972', 'https://old.reddit.com/r/walkaway/comments/x2gmkp/how_to_charge_your_electric_vehicle_in_california/imk0keh/', 'Its not a mistake. Theyre wrecking everything under the guise of "climate change".', '0.2584', 38])

def test_2_check_score_two_msg_2(data):
    """
    Test 2
    """
    assert data.check_score(0) == (['comment', 'imlddn9', '2qh3l', 'news', 'false', '1661990368', 'https://old.reddit.com/r/news/comments/x2cszk/us_life_expectancy_down_for_secondstraight_year/imlddn9/', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', '0.5719', 2], 
                                   ['comment', 'imkvel6', '2qh6c', 'christianity', 'false', '1661982640', 'https://old.reddit.com/r/Christianity/comments/x2d3g0/major_evangelical_organization_says_christianity/imkvel6/', "You are correct. I have no concern for climate change. I leave that in God's hands to not man's.\n\nHow would you express Christ to the world?", '-0.34', -2])

def test_3_check_score_two_msg_3(data):
    """
    Test 3
    """
    assert data.check_score(500) == (['comment', 'imlddn9', '2qh3l', 'news', 'false', '1661990368', 'https://old.reddit.com/r/news/comments/x2cszk/us_life_expectancy_down_for_secondstraight_year/imlddn9/', 'Yeah but what the above commenter is saying is their base doesn’t want any of that. They detest all of those things, even the small gradual changes. Investing in nuclear energy is a tacit acknowledgement of man made climate change. Any acknowledgement or concession and they will be primaried out in a minute', '0.5719', 2], 
                                   ['comment', 'im9i19f', '2qh3l', 'news', 'false', '1661785239', 'https://old.reddit.com/r/news/comments/x0fabf/last_member_of_indigenous_tribe_dies_in_brazil/im9i19f/', 'It’s literally the one reason we will wipe ourselves out via climate change. We want money. It’s that simple. And no local government on earth will chose long term survival over corporate profits.', '0.25', 498])

def test_4_check_score_fast_two_msg_1(data):
    """
    Test 4
    """
    assert data.check_score_fast(40) == (['comment', 'c0i28hb', '2qh1n', 'environment', 'false', '1262383373', 'https://old.reddit.com/r/environment/comments/akl0m/is_the_airborne_fraction_of_anthropogenic_co2/c0i28hb/', 'No one truly knows what\'s going on, but for a while there, I was thinking, "Duh, hello, of course it\'s us."  Some information I\'ve seen recently leads me to wonder.  Also, the amount of CO2 we pump into the air is great already and has not really changed year to year.  Why the sudden, accelerated climb?  Is the "feedback loop" really taking hold so quickly?  Did we just have to get it to a certain point and now it will skyrocket?  I see posts on reddit every other week about some finding or another that says warming is happening even faster than previously thought.  \n\nSomething else that has me wondering:\n\nPeople talk about the methane being released by creatures trapped under what should be permafrost, which is now melting.  What was going on with the climate when it was warm enough for those creatures to be alive and thriving in their environments?\n\nI\'m also not trying to say climate change and global warming aren\'t happening, and that the results won\'t be catastrophic, but I just wonder about the answers to some of these questions.', '0.9366', 2], 
                                         ['comment', 'c0kk63f', 'mouw', 'science', 'false', '1266186173', 'https://old.reddit.com/r/science/comments/b20b8/im_a_global_warming_skeptic_and_i_invite_you_to/c0kk63f/', "Infodump incoming:\n\nIn regards to models:\n\nJames Hansen, in his 1988 paper correctly predicted, using a model, confirmed by his follow-up 2005 report\n\nGlobal Climate Changes as Forecast by the Goddard Institute for Space Studies Three-Dimensional Model, J. Hansen, I. Fung, A. Lacis, D. Rind, S. Lebedeff, R. Ruedy and G. Russell, Journal of Geophysical Research Vol. 93. No. D8, Pages 9341-9364, August 1988.\n\nGlobal temperature change, James Hansen, Makiko Sat, Reto Ruedy, Ken Lo, David W. Lea, and Martin Medina-Elizade, Proceedings of the National Academy of Sciences of the USA, vol.103 no.39, September 2006\n\n&gt;Our climate model, driven mainly by increasing human-made greenhouse gases and aerosols among other forcings, calculates that Earth is now absorbing 0.85 ± 0.15 W/m2 more energy from the Sun than it is emitting to space. This imbalance is confirmed by precise measurements of increasing ocean heat content over the past 10 years. Implications include: (i) expectation of additional global warming of about 0.6°C without further change of atmospheric composition; (ii) confirmation of the climate system’s lag in responding to forcings, implying the need for anticipatory actions to avoid any specified level of climate change; and (iii) likelihood of acceleration of ice sheet disintegration and sea level rise.\n\nEarth’s Energy Imbalance: Confirmation and Implications, James Hansen, Larissa Nazarenko, Reto Ruedy, Makiko Sato, Josh Willis, Anthony Del Genio, Dorothy Koch, Andrew Lacis, Ken Lo, Surabi Menon, Tica Novakov, Judith Perlwitz, Gary Russell, Gavin A. Schmidt, Nicholas Tausnev, 28 April 2004 / Page 1/ 10.1126/science.1110252\n\nIn regards to Carbon Dioxide and its link to global warming:\n\n&gt;The evolution of the Earth's climate has been extensively studied and a strong link between increases in surface temperatures and greenhouse gases has been established\n\n&gt;...\n\n&gt;Our results provide direct experimental evidence for a significant increase in the Earth's greenhouse effect that is consistent with concerns over radiative forcing of climate.\n\nIncreases in greenhouse forcing inferred from the outgoing longwave radiation spectra of the Earth in 1970 and 1997, John E. Harries, Helen E. Brindley, Pretty J. Sagoo &amp; Richard J. Bantges, Nature 410, 355-357,March 2001, doi:10.1038/35066553\n\n&gt;Measurements of spectrally resolved outgoing longwave radiation allows signatures of many aspects of greenhouse warming to be distinguished...\n\n&gt;...\n\n&gt;This provides direct evidence for signicant changes in the greenhouse gases over the last 34 years, consistent with concerns over the changes in radiative forcing of the climate.\n\nComparison of spectrally resolved outgoing longwave data between 1970 and present, Jennifer A. Griggs and John E. Harries, Imperial College\n\n&gt;Previously published work using satellite observations of the clear sky infrared emitted radiation by the Earth in 1970, 1997 and in 2003 showed the appearance of changes in the outgoing spectrum, which agreed with those expected from known changes in the concentrations of well-mixed greenhouse gases over this period. Thus, the greenhouse forcing of the Earth has been observed to change in response to these concentration changes.\n\nSpectral signatures of climate change in the Earth’s infrared spectrum between 1970 and 2006, Claudine Chen, John Harries, Helen Brindley, Mark Ringer, Imperial College\n\nSpecial Sensor Microwave Imager results and analysis:\n\n&gt;Data from the satellite-based Special Sensor Microwave Imager (SSM/I) show that the total atmospheric moisture content over oceans has increased by 0.41 kg/m2 per decade since 1988. Results from current climate models indicate that water vapor increases of this magnitude cannot be explained by climate noise alone. In a formal detection and attribution analysis using the pooled results from 22 different climate models, the simulated “fingerprint” pattern of anthropogenically caused changes in water vapor is identifiable with high statistical confidence in the SSM/I data. Experiments in which forcing factors are varied individually suggest that this fingerprint “match” is primarily due to human-caused increases in greenhouse gases and not to solar forcing or recovery from the eruption of Mount Pinatubo. Our findings provide preliminary evidence of an emerging anthropogenic signal in the moisture content of earth's atmosphere.\n\nIdentification of human-induced changes in atmospheric moisture content, B. D. Santera, C. Mearsc, F. J. Wentzc, K. E. Taylora, P. J. Glecklera, T. M. L. Wigleyd, T. P. Barnette, J. S. Boylea, W. Brüggemannf, N. P. Gillettg, S. A. Kleina, G. A. Meehld, T. Nozawah, D. W. Piercee, P. A. Stotti, W. M. Washingtond, and M. F. Wehnerj. Proceeding of the National Academy of Science of the United States of America vol. 104 no. 39, September 25, 2007, 15248-15253, 10.1073/pnas.0702872104\n\nI will provide more papers on subjects as request.", '0.988', 38])

def test_5_check_score_fast_two_msg_2(data):
    """
    Test 5
    """
    assert data.check_score_fast(0) == (['comment', 'c0i28hb', '2qh1n', 'environment', 'false', '1262383373', 'https://old.reddit.com/r/environment/comments/akl0m/is_the_airborne_fraction_of_anthropogenic_co2/c0i28hb/', 'No one truly knows what\'s going on, but for a while there, I was thinking, "Duh, hello, of course it\'s us."  Some information I\'ve seen recently leads me to wonder.  Also, the amount of CO2 we pump into the air is great already and has not really changed year to year.  Why the sudden, accelerated climb?  Is the "feedback loop" really taking hold so quickly?  Did we just have to get it to a certain point and now it will skyrocket?  I see posts on reddit every other week about some finding or another that says warming is happening even faster than previously thought.  \n\nSomething else that has me wondering:\n\nPeople talk about the methane being released by creatures trapped under what should be permafrost, which is now melting.  What was going on with the climate when it was warm enough for those creatures to be alive and thriving in their environments?\n\nI\'m also not trying to say climate change and global warming aren\'t happening, and that the results won\'t be catastrophic, but I just wonder about the answers to some of these questions.', '0.9366', 2], 
                                        ['comment', 'c0ibd6w', 'mouw', 'science', 'false', '1262805680', 'https://old.reddit.com/r/science/comments/amcj7/global_warming_denialists_holy_crap_its_cold_in/c0ibd6w/', "Two Decades Tops\n\nThe general consensus is that Al Gore's fury will be unleashed via massive global climate change sometime between tomorrow and 20 years from now...or not at all. \n\n", '-0.1027', -2])
    
def test_6_check_score_fast_two_msg_3(data):
    """
    Test 6
    """
    assert data.check_score_fast(500) == (['comment', 'c0i28hb', '2qh1n', 'environment', 'false', '1262383373', 'https://old.reddit.com/r/environment/comments/akl0m/is_the_airborne_fraction_of_anthropogenic_co2/c0i28hb/', 'No one truly knows what\'s going on, but for a while there, I was thinking, "Duh, hello, of course it\'s us."  Some information I\'ve seen recently leads me to wonder.  Also, the amount of CO2 we pump into the air is great already and has not really changed year to year.  Why the sudden, accelerated climb?  Is the "feedback loop" really taking hold so quickly?  Did we just have to get it to a certain point and now it will skyrocket?  I see posts on reddit every other week about some finding or another that says warming is happening even faster than previously thought.  \n\nSomething else that has me wondering:\n\nPeople talk about the methane being released by creatures trapped under what should be permafrost, which is now melting.  What was going on with the climate when it was warm enough for those creatures to be alive and thriving in their environments?\n\nI\'m also not trying to say climate change and global warming aren\'t happening, and that the results won\'t be catastrophic, but I just wonder about the answers to some of these questions.', '0.9366', 2], 
                                     ['comment', 'da1cseo', '2cneq', 'politics', 'false', '1479224805', 'https://old.reddit.com/r/politics/comments/5d2t6i/we_need_to_talk_about_the_online_radicalisation/da1cseo/', "I'll be honest about my perspective knowing I'll just get shit on. The Donald should have been quarantined a long time ago and after the victory I think I have had enough of Reddit. The absolute denial of climate change and the aggressive hate on any opposed viewpoint is just too much to see everyday.", '-0.6808', 498])

Overwriting test_data.py


In [ ]:
!pytest . -vv

Test session starts (platform: linux, Python 3.7.14, pytest 3.6.4, pytest-sugar 0.9.5)
cachedir: .pytest_cache
rootdir: /content, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_data.py::test_1_check_score_two_msg_1 ✓                     17% █▋        
 test_data.py::test_2_check_score_two_msg_2 ✓                     33% ███▍      
 test_data.py::test_3_check_score_two_msg_3 ✓                     50% █████     
 test_data.py::test_4_check_score_fast_two_msg_1 ✓                67% ██████▋   
 test_data.py::test_5_check_score_fast_two_msg_2 ✓                83% ████████▍ 
 test_data.py::test_6_check_score_fast_two_msg_3 ✓               100% ██████████

Results (161.72s):
       6 passed
